In [1]:
# Imports here
import torch
import PIL
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torchvision import datasets, transforms, models
from torch import nn
from torch import optim
from collections import OrderedDict
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# data_dir = '/kaggle/input/flower-classification'
# train_dir = data_dir + '/train'
# valid_dir = data_dir + '/valid'
# test_dir = data_dir + '/test'

In [ ]:

data_dir = '/kaggle/input/flower-classification'
train_dir = data_dir + '/train'

# Define your transformations
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load all the images from the train folder
all_data = datasets.ImageFolder(train_dir, transform=train_transforms)

# Calculate the sizes for train, validation, and test sets
total_size = len(all_data)
train_size = int(0.7 * total_size)
test_size = int(0.2 * total_size)
valid_size = total_size - train_size - test_size

# Use random_split to split the dataset
train_data, valid_data, test_data = torch.utils.data.random_split(all_data, [train_size, valid_size, test_size])

# Create data loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=50, shuffle=False)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=50)
testloader = torch.utils.data.DataLoader(test_data, batch_size=50)


In [ ]:
# Load a pre-trained network 
model = models.vgg16(pretrained=True)
model.name = "vgg16"
model

In [ ]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# Define a new, untrainted feed-forward network as a classifier, using ReLU activations and dropout
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096, bias=True)),
                          ('relu1', nn.ReLU()),
                          ('dropout1', nn.Dropout(p=0.5)),
                          ('fc2', nn.Linear(4096, 14, bias=True)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [ ]:
# Device agnostic code, automatically uses CUDA if it's enabled
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# change to device
model.to(device)

In [ ]:
# Define loss and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Define deep learning method
epochs = 5
print_every = 30 # Prints every 30 images out of batch of 50 images
steps = 0

In [ ]:
# Implement a function for the validation pass
def validation(model, testloader, criterion):
    test_loss = 0
    accuracy = 0
    
    for ii, (inputs, labels) in enumerate(testloader):
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        output = model.forward(inputs)
        test_loss += criterion(output, labels).item()
        
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [ ]:
print(model)

In [ ]:
# Train the classifier layers using backpropogation using the pre-trained network to get features

print("Training process initializing .....\n")

for e in range(epochs):
    running_loss = 0
    model.train() # Technically not necessary, setting this for good measure
    
    for ii, (inputs, labels) in enumerate(trainloader):
        steps += 1
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward and backward passes
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()

            with torch.no_grad():
                valid_loss, accuracy = validation(model, validloader, criterion)
            
            print("Epoch: {}/{} | ".format(e+1, epochs),
                  "Training Loss: {:.4f} | ".format(running_loss/print_every),
                  "Validation Loss: {:.4f} | ".format(valid_loss/len(validloader)),
                  "Validation Accuracy: {:.4f}".format(accuracy/len(validloader)))
            
            running_loss = 0
            model.train()

print("\nTraining process is now complete!!")

In [ ]:
all_labels = []
all_predictions = []
from time import time
correct = 0
total = 0
start_time = time()
with torch.no_grad():
    model.eval()
    for images, labels in testloader:
        all_labels.extend(labels.numpy())
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predicted_tensor_cpu = predicted.to('cpu')
        all_predictions.extend(predicted_tensor_cpu.numpy())
end_time = time()
print("Time: ",end_time - start_time)

print('Accuracy achieved by the network on test images is: %d%%' % (100 * correct / total))

In [ ]:
print(len(all_labels))
print(len(all_predictions))

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)
# Calculate the confusion matrix
cm = confusion_matrix(all_labels, all_predictions)
print("Confusion Matrix:")
print(cm)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(all_labels, all_predictions)
print(report)

In [ ]:
# import requests

# url = "https://raw.githubusercontent.com/pytorch/pytorch/main/torch/utils/collect_env.py"
# response = requests.get(url)

# # Check if the request was successful (status code 200)
# if response.status_code == 200:
#     # Save the content to a file
#     with open("collect_env.py", "wb") as f:
#         f.write(response.content)
#         print("done")
# else:
#     print(f"Failed to download the script. Status code: {response.status_code}")



In [ ]:
#!python collect_env.py

To Create a checkpoint [Reference](https://discuss.pytorch.org/t/saving-and-loading-a-model-in-pytorch/2610)

In [ ]:
# Assuming your train_data is a Subset object
original_dataset = train_data.dataset

# Create a class_to_idx mapping
model.class_to_idx = {class_name: idx for idx, class_name in enumerate(original_dataset.classes)}


In [ ]:
print(model.class_to_idx)

In [ ]:
checkpoint = {'architecture': model.name,
             'classifier': model.classifier,
             'class_to_idx': model.class_to_idx,
             'state_dict': model.state_dict()}

torch.save(checkpoint, 'model.pth')

In the next cell, in the load checkpoint function change the models.vgg() to other models

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
# Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint():
    """
    Loads deep learning model checkpoint.
    """
    
    # Load the saved file
    checkpoint = torch.load("/kaggle/working/model.pth")
    
    # Download pretrained model
    model = models.vgg16(pretrained=True);
    
    # Freeze parameters so we don't backprop through them
    for param in model.parameters(): param.requires_grad = False
    
    # Load stuff from checkpoint
    model.class_to_idx = checkpoint['class_to_idx']
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])

    
    return model

In [ ]:
# wget https://raw.githubusercontent.com/pytorch/pytorch/main/torch/utils/collect_env.py
# python collect_env.py

# QUANTIZATION

In [ ]:
model_fp16 = load_checkpoint()

In [ ]:
# weights = model.state_dict()
# print(weights)

In [ ]:
model_fp16.half()

In [ ]:
weights_half = model_fp16.state_dict()
print(weights_half['features.0.weight'].dtype)

In [ ]:
#NOT WORKING
# new_test = testloader
# for inputs, targets in new_test:
#     inputs = inputs.half()

In [ ]:
model_fp16.to(device)

In [ ]:
all_labels_fp16 = []
all_predictions_fp16 = []

In [ ]:
correct_fp16 = 0
total_fp16 = 0
start_time_fp16 = time()
with torch.no_grad():
    model_fp16.eval()
    for images, labels in testloader:
        all_labels_fp16.extend(labels.numpy())
        images, labels = images.to(device), labels.to(device)
        outputs = model_fp16(images.half())
        _, predicted = torch.max(outputs.data, 1)
        total_fp16 += labels.size(0)
        correct_fp16 += (predicted == labels).sum().item()
        predicted_tensor_cpu = predicted.to('cpu')
        all_predictions_fp16.extend(predicted_tensor_cpu.numpy())
end_time_fp16 = time()
print("Time: ",end_time_fp16 - start_time_fp16)

print('Accuracy achieved by the network on test images is: %d%%' % (100 * correct_fp16 / total_fp16))

In [ ]:
all_labels_fp16 = np.array(all_labels_fp16)
all_predictions_fp16 = np.array(all_predictions_fp16)
# Calculate the confusion matrix
cm_fp16 = confusion_matrix(all_labels_fp16, all_predictions_fp16)
print("Confusion Matrix:")
print(cm_fp16)

In [ ]:
report_new = classification_report(all_labels_fp16, all_predictions_fp16)
print(report_new)

In [ ]:
model_fp16.class_to_idx = {class_name: idx for idx, class_name in enumerate(original_dataset.classes)}

In [ ]:
model_fp16.name= 'vgg16'

In [ ]:
checkpoint_fp16 = {'architecture': model_fp16.name,
             'classifier': model_fp16.classifier,
             'class_to_idx': model_fp16.class_to_idx,
             'state_dict': model_fp16.state_dict()}

torch.save(checkpoint_fp16, 'model_fp16.pth')

# -----------------------------------------------------

In [ ]:
model_64 = load_checkpoint()
model_64.double()

In [ ]:
weights_64 = model_64.state_dict()
print(weights_64['features.0.weight'].dtype)

In [ ]:
model_64.to(device)
all_labels_fp64 = []
all_predictions_fp64 = []
correct_fp64 = 0
total_fp64 = 0
start_time_fp64 = time()
with torch.no_grad():
    model_64.eval()
    for images, labels in testloader:
        all_labels_fp64.extend(labels.numpy())
        images, labels = images.to(device), labels.to(device)
        outputs = model_64(images.double())
        _, predicted = torch.max(outputs.data, 1)
        total_fp64 += labels.size(0)
        correct_fp64 += (predicted == labels).sum().item()
        predicted_tensor_cpu = predicted.to('cpu')
        all_predictions_fp64.extend(predicted_tensor_cpu.numpy())
end_time_fp64 = time()
print("Time: ",end_time_fp64 - start_time_fp64)

print('Accuracy achieved by the network on test images is: %d%%' % (100 * correct_fp64 / total_fp64))

In [ ]:
all_labels_fp64 = np.array(all_labels_fp64)
all_predictions_fp64 = np.array(all_predictions_fp64)
# Calculate the confusion matrix
cm_fp64 = confusion_matrix(all_labels_fp64, all_predictions_fp64)
print("Confusion Matrix:")
print(cm_fp64)

In [ ]:
report_64 = classification_report(all_labels_fp64, all_predictions_fp64)
print(report_64)

In [ ]:
model_64.class_to_idx = {class_name: idx for idx, class_name in enumerate(original_dataset.classes)}
model_64.name= 'vgg16'

In [ ]:
checkpoint_fp64 = {'architecture': model_64.name,
             'classifier': model_64.classifier,
             'class_to_idx': model_64.class_to_idx,
             'state_dict': model_64.state_dict()}

torch.save(checkpoint_fp64, 'model_64.pth')

# ------------------------------------------------------

# Dont trust anything below this 

In [ ]:
import copy
from torch.ao.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import convert_fx, prepare_fx

In [ ]:
fp32_model = load_checkpoint()

In [ ]:
quantized_model.class_to_idx = {class_name: idx for idx, class_name in enumerate(original_dataset.classes)}
quantized_model.name = 'vgg16'
checkpoint = {'architecture': quantized_model.name,
             'classifier': quantized_model.classifier,
             'class_to_idx': quantized_model.class_to_idx,
             'state_dict': quantized_model.state_dict()}

torch.save(checkpoint, 'quantized_model.pth')

In [ ]:
model_int8 = load_checkpoint()
model_int8.to(device)
model_int8.eval()

model_int8.qconfig = torch.quantization.default_qconfig
torch.quantization.prepare(model_int8, inplace=True)

print('\nPost Training Quantization Prepare: Inserting Observers by Calibrate')
_,accuracy = validation(model_int8, validloader, criterion)
print("Calibrate done")

# Convert to quantized model
torch.quantization.convert(model_int8, inplace=True)
print('Post Training Quantization: Convert done')


print('\n After quantization: \n',model_int8)

In [ ]:
print(accuracy)

In [ ]:
# model_fp32 = load_checkpoint()
# model_fp32.to('cpu')
# model_fp32.eval()
# model_fp32.qconfig = torch.ao.quantization.get_default_qconfig('x86')
# model_fp32_prepared = torch.ao.quantization.prepare(model_fp32)
# input_fp32 = torch.randn(1, 3, 224, 224)
# model_fp32_prepared(input_fp32)
# model_int8 = torch.ao.quantization.convert(model_fp32_prepared)

In [ ]:
# model_int8.state_dict()['features.0.weight'].dtype

In [ ]:
model_int8.class_to_idx = {class_name: idx for idx, class_name in enumerate(original_dataset.classes)}
model_int8.name = 'vgg16'

In [ ]:
checkpoint = {'architecture': model_int8.name,
             'classifier': model_int8.classifier,
             'class_to_idx': model_int8.class_to_idx,
             'state_dict': model_int8.state_dict()}

torch.save(checkpoint, 'model_int8.pth')

In [ ]:
all_labels_int8 = []
all_predictions_int8 = []

In [ ]:
model_int8.to('cpu')

In [ ]:
!pip install torch -U


In [ ]:
import torch

In [ ]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu110/torch_stable.html


In [ ]:
all_labels_int8 = []
all_predictions_int8 = []
correct_int8 = 0
total_int8 = 0
start_time_int8 = time()
with torch.no_grad():
    model_int8.eval()
    for images, labels in testloader:
        all_labels_int8.extend(labels.numpy())
        #images, labels = images.to(device), labels.to(device)
        images = torch.quantize_per_tensor(images, scale=1.0, zero_point=0, dtype=torch.qint8)
        outputs = model_int8(images)
        _, predicted = torch.max(outputs.data, 1)
        total_int8 += labels.size(0)
        correct_int8 += (predicted == labels).sum().item()
        #predicted_tensor_cpu = predicted.to('cpu')
        all_predictions_int8.extend(predicted.numpy())
end_time_int8 = time()
print("Time: ",end_time_int8 - start_time_int8)

print('Accuracy achieved by the network on test images is: %d%%' % (100 * correct_int8 / total_int8))

In [ ]:
myModel.state_dict()['features.0.weight']


In [ ]:
myModel.class_to_idx = {class_name: idx for idx, class_name in enumerate(original_dataset.classes)}

In [ ]:
myModel.name = 'vgg16'

In [ ]:
checkpoint = {'architecture': myModel.name,
             'classifier': myModel.classifier,
             'class_to_idx': myModel.class_to_idx,
             'state_dict': myModel.state_dict()}

torch.save(checkpoint, 'model_int8.pth')

## INT8

In [ ]:
def validation(model, testloader, criterion):
    test_loss = 0
    accuracy = 0
    
    for ii, (inputs, labels) in enumerate(testloader):
        
        inputs, labels = inputs.to(device), labels.to(device)
        
        output = model.forward(inputs)
        test_loss += criterion(output, labels).item()
        
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [ ]:
import torchvision.transforms as transforms
import torch.quantization


In [ ]:
model_fp32 = load_checkpoint()

In [ ]:
model_fp32.to(device)

In [ ]:
input_data = next(iter(trainloader))[0][:1]  
input_data = input_data.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Quantize the input data to qint8
quantized_input_data = torch.quantize_per_tensor(input_data, scale=1.0, zero_point=0, dtype=torch.qint8)

In [ ]:
model_int8 = torch.quantization.QuantWrapper(model_fp32)
model_int8.qconfig = torch.quantization.default_qconfig
torch.quantization.prepare(model_int8, inplace=True)

In [ ]:
with torch.no_grad():
    model_int8(input_data)

In [ ]:
torch.quantization.convert(model_int8, inplace=True)

In [ ]:
int_weights = model_int8.state_dict()

print(int_weights['module.features.0.weight'].dtype)

In [ ]:
model_int8.state_dict()

In [ ]:
all_labels_int8 = []
all_predictions_int8 = []

In [ ]:
model_int8.to('cpu')

In [ ]:
correct_pred = 0
total_pred = 0
start_time_int8 = time()
with torch.no_grad():
    model_int8.eval()
    for images,labels in testloader:
        all_labels_int8.extend(labels.numpy())
        images, labels = images.to(device), labels.to(device)
        outputs = model_int8(images)
        _, predicted = torch.max(outputs.data, 1)
        total_pred += labels.size(0)
        correct_pred += (predicted == labels).sum().item()
        predicted_tensor_cpu = predicted.to('cpu')
        all_predictions_int8.extend(predicted_tensor_cpu.numpy())
end_time_int8 = time()
print("Time: ",end_time_int8 - start_time_int8)

print('Accuracy achieved by the network on test images is: %d%%' % (100 * correct_pred / total_pred))

In [ ]:
all_labels_int8 = np.array(all_labels_int8)
all_predictions_int8 = np.array(all_predictions_int8)

# Calculate the confusion matrix
cm_int8 = confusion_matrix(all_labels_int8, all_predictions_int8)

print("Confusion Matrix:")
print(cm_int8)

In [ ]:
report_int8 = classification_report(all_labels_int8, all_predictions_int8)
print(report_int8)